### In this project I am using sqlite to answer some business questions about a fictitious digital media store represented in the Chinook database.

### About Chinook database: The Chinook data model represents a digital media store, including tables for artists, albums, media tracks, invoices and customers.

### Connecting to Chinook database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

### Getting general information about tables in the database

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Which Genres are the most sold in USA?
### Rock, Alternative & Punk and Metal are the most sold genres

In [3]:
%%sql
WITH units_sold_usa AS
    (
        SELECT
            t.track_id,
            t.genre_id
        FROM invoice as i
            INNER JOIN invoice_line AS il ON i.invoice_id = il.invoice_id
            INNER JOIN track AS t ON il.track_id = t.track_id
        WHERE i.billing_country = "USA" 
    )
    
SELECT 
    g.name as genre_name,
    COUNT (usu.track_id) AS units_sold,
    ROUND ( CAST ( COUNT (usu.track_id) AS FLOAT) / (SELECT COUNT(*) FROM units_sold_usa ), 2 ) AS pct
FROM units_sold_usa AS usu
    INNER JOIN genre AS g ON usu.genre_id = g.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;



 * sqlite:///chinook.db
Done.


genre_name,units_sold,pct
Rock,561,0.53
Alternative & Punk,130,0.12
Metal,124,0.12
R&B/Soul,53,0.05
Blues,36,0.03
Alternative,35,0.03
Pop,22,0.02
Latin,22,0.02
Hip Hop/Rap,20,0.02
Jazz,14,0.01


## Sales support agents performance comparison
### The performance of sales support agents are similiar, the 10% sales difference between each agent is proportional to # of customers assigned to them and their hire dates.

In [4]:
%%sql
WITH customer_totals AS
    ( SELECT
        c.customer_id,
        ROUND (SUM(i.total), 2) AS total,
         c.support_rep_id
    FROM customer AS c
        INNER JOIN invoice AS i ON i.customer_id = c.customer_id
    GROUP BY 1
    )
        
SELECT
    e.first_name || " " || e.last_name AS rep_name,
    ROUND( SUM(ct.total), 2) AS total,
    COUNT (e.employee_id )AS n_customer,
    e.hire_date AS employee_hire_date
FROM employee AS e
INNER JOIN customer_totals AS ct ON e.employee_id = ct.support_rep_id
GROUP BY 1
ORDER BY 2  DESC;


 * sqlite:///chinook.db
Done.


rep_name,total,n_customer,employee_hire_date
Jane Peacock,1731.51,21,2017-04-01 00:00:00
Margaret Park,1584.0,20,2017-05-03 00:00:00
Steve Johnson,1393.92,18,2017-10-17 00:00:00


## Sales Data by Country
### Sales, number of customers, average order size and average customer lifetime values for each country.
### Since there are many countries with only one customer they are classified as "Other"

In [5]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       i.*
     FROM invoice AS i
     INNER JOIN customer AS c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) AS customers,
        ROUND (SUM(total), 2 ) AS total_sales,
        ROUND( SUM(total) / count(distinct customer_id), 2) AS customer_lifetime_value,
        ROUND( SUM(total) / count(distinct invoice_id), 2) AS average_order,
        
-- Using CASE to sort 'Others' category on the bottom -- 
        
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

 * sqlite:///chinook.db
Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.49,7.94,80.04
Canada,8,535.59,7.05,66.95
Brazil,5,427.68,7.01,85.54
France,5,389.07,7.78,77.81
Germany,4,334.62,8.16,83.66
Czech Republic,2,273.24,9.11,136.62
United Kingdom,3,245.52,8.77,81.84
Portugal,2,185.13,6.38,92.57
India,2,183.15,8.72,91.58
Other,15,1094.94,7.45,73.0


## Percentage of album purchases vs non-album purchases
### We see that majority (81%) of purchases are non-album purchases

In [23]:
%%sql
-- Explanation of 2 subqueries before and after AND clause: --
-- If the minimum track id from an invoice is the same as the first track in the invoice then it is considered an album purchase --
-- This is done to make sure they are the same, for example set (abcd) EXCEPT (abc) gives (d) whereas (abc) EXCEPT (abcd) gives NULL --
-- we want to pass the cases only when (abc) EXCEPT (abc) -> null and not (abc) EXCEPT (abcd) -> null -- 
-- to check this we also check (abcd) EXCEPT (abc) -> (d) therefore we use AND in between two reversed subqueries -- 


WITH invoice_min_track AS
    (
     SELECT
         il.invoice_id AS invoice_id,
         MIN(il.track_id) AS min_track_id
     FROM invoice_line il
     GROUP BY invoice_id
    )

SELECT
    album_purchase,
    COUNT(invoice_id) AS number_of_invoices,
    ROUND( 
        CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                          ), 2 
        ) AS percent
FROM
    (
    SELECT
        imt.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track AS t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track AS t2
                                      WHERE t2.track_id = imt.min_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line AS il2
                  WHERE il2.invoice_id = imt.invoice_id
                 ) IS NULL
                -- above subquery is in regular order --
             AND -- Here we use AND between two subqueries to make sure they both represent same boolean value --
                
                -- Below subquery is in reversed order of the above one -- 
                (
                  SELECT il2.track_id FROM invoice_line AS il2
                  WHERE il2.invoice_id = imt.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track AS t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track AS t2
                                      WHERE t2.track_id = imt.min_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_min_track AS imt
    )
GROUP BY album_purchase;



 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent
no,500,0.81
yes,114,0.19
